In [1]:
import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sample = (" Authorities in Turkey -- who say Khashoggi"
          "was murdered by a hit squad sent from Riyadh"
          "-- are still searching for the journalist's remains."
          " Earlier this week, the chief prosecutor's office "
          "said Khashoggi's body was dismembered after he was str"
          "angled, while the Washington Post reported investigators"
          "were looking into the theory that the body was dissolved "
          "in acid. A source close to the Saudi Royal Palace has de"
          "nied any knowledge of the body's whereabouts.")
idx2char = list(set(sample))
char2idx = {c:i for i,c in enumerate(idx2char)}

In [3]:
input_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sample)-sequence_length):
    x_str = sample[i:i+sequence_length]
    y_str = sample[i+1:i+sequence_length+1]
    x = [char2idx[c] for c in x_str]
    y = [char2idx[c] for c in y_str]
    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

X = tf.placeholder(tf.int32, shape=[None,sequence_length])
Y = tf.placeholder(tf.int32, shape=[None,sequence_length])
x_one_hot = tf.one_hot(X, input_dim)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state,
                                    dtype=tf.float32)
X_for_fc = tf.reshape(outputs, [-1,hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
outputs = tf.reshape(outputs, [batch_size,sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,
                                                weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)


In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2000):
    l,_ = sess.run([loss,train], feed_dict={X:dataX, Y:dataY})
    if step % 100 == 0:
#         result = sess.run(prediction, feed_dict={X:x_data})
#         result_str = [idx2char[c] for c in np.squeeze(result)]
        print ("step:",step,"Loss:",l)

step: 0 Loss: 3.6138318
step: 100 Loss: 0.32329622
step: 200 Loss: 0.3110854
step: 300 Loss: 0.31010175
step: 400 Loss: 0.308718
step: 500 Loss: 0.30951244
step: 600 Loss: 0.30801818
step: 700 Loss: 0.30803755
step: 800 Loss: 0.30811808
step: 900 Loss: 0.3080096
step: 1000 Loss: 0.3083549
step: 1100 Loss: 0.30769554
step: 1200 Loss: 0.30764076
step: 1300 Loss: 0.30837834
step: 1400 Loss: 0.3075836
step: 1500 Loss: 0.30759314
step: 1600 Loss: 0.30761254
step: 1700 Loss: 0.30759156
step: 1800 Loss: 0.3076332
step: 1900 Loss: 0.30749762


# RNN 셀을 여러개 쌓아보기

In [1]:
import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sample = (" Authorities in Turkey -- who say Khashoggi"
          "was murdered by a hit squad sent from Riyadh"
          "-- are still searching for the journalist's remains."
          " Earlier this week, the chief prosecutor's office "
          "said Khashoggi's body was dismembered after he was str"
          "angled, while the Washington Post reported investigators"
          "were looking into the theory that the body was dissolved "
          "in acid. A source close to the Saudi Royal Palace has de"
          "nied any knowledge of the body's whereabouts.")
idx2char = list(set(sample))
char2idx = {c:i for i,c in enumerate(idx2char)}

In [3]:
input_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sample)-sequence_length):
    x_str = sample[i:i+sequence_length]
    y_str = sample[i+1:i+sequence_length+1]
    x = [char2idx[c] for c in x_str]
    y = [char2idx[c] for c in y_str]
    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

In [4]:
X = tf.placeholder(tf.int32, shape=[None,sequence_length])
Y = tf.placeholder(tf.int32, shape=[None,sequence_length])
X_one_hot = tf.one_hot(X, input_dim)

def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
# 세개의 lstm 셀층을 더만든다
multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell()] * 3, state_is_tuple=True)

outputs, _status = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets=Y, weights=weights)

loss =tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)



In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2000):
    l,_ = sess.run([loss,train], feed_dict={X:dataX, Y:dataY})
    if step % 100 == 0:
#         result = sess.run(prediction, feed_dict={X:x_data})
#         result_str = [idx2char[c] for c in np.squeeze(result)]
        print ("step:",step,"Loss:",l)

step: 0 Loss: 3.6112285
step: 100 Loss: 0.88494325
step: 200 Loss: 0.34947336
step: 300 Loss: 0.31666362
step: 400 Loss: 0.31217918
step: 500 Loss: 0.31223598
step: 600 Loss: 0.30946738
step: 700 Loss: 0.30904177
step: 800 Loss: 0.30906898
step: 900 Loss: 0.30864483
step: 1000 Loss: 0.30929884
step: 1100 Loss: 0.30907312
step: 1200 Loss: 0.308635
step: 1300 Loss: 0.30810642
step: 1400 Loss: 0.30807576
step: 1500 Loss: 0.30798376
step: 1600 Loss: 0.308184
step: 1700 Loss: 0.30927673
step: 1800 Loss: 0.30779734
step: 1900 Loss: 0.3078444


In [8]:
result = sess.run(outputs, feed_dict={X:dataX})
for j, result in enumerate(result):
    index = np.argmax(result, axis=1)
    if j is 0:
        print(''.join([idx2char[t] for t in index]), end='')
    else:
        print(idx2char[index[-1]], end='')

suthorities in Turkey -- who say Khashoggi'as murdered by a hit squad sent from Riyadh-- are still searching for the journalist's remains. Earlier this week, the chief prosecutor's office said Khashoggi's body was dissembered after he was strangled, while the Washington Post reported investigatorswere looking into the theory that the body was dissolved in acid. A source close to the Saudi Royal Palace has denied any knowledge of the body's whereabouts.